In [ ]:
!pip3.13 install ragas --break-system-packages

In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from langchain_ollama.llms import OllamaLLM
from ragas.llms import LangchainLLMWrapper

In [ ]:
import pandas as pd

In [ ]:
from ragas import SingleTurnSample, EvaluationDataset, evaluate

In [ ]:
from ragas.run_config import RunConfig

In [ ]:
csv_df = pd.read_csv('/Users/user/Files/DatasetD_eval.csv', sep=';').dropna()

In [ ]:
csv_df

,Question/Prompt,Ground Truth,Contexts,Answer
0,Suggest me a Collect service for reading the\n...,CSV data retrieval,[Document(metadata={'source': '/Users/massimil...,"Based on the provided context, you can use the..."
1,Suggest me a Monitor service for training a\nm...,Multiple Linear Regression Service,"{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""ti...","Based on your requirements, here's a suggested..."
2,Suggest me a Collect service for reading the c...,CSV data retrieval,"{\n ""openapi"": ""3.1.0"",\n ""info"": {\n ""ti...","Based on the provided context, you can use the..."
3,Suggest me a Dispatch service for applying ano...,Pandas DataFrame Anonymization Service,"{\n ""openapi"": ""3.0.0"",\n ""info"": {\n ""ti...","Based on the provided context, you can use the..."


In [ ]:
evaluation_samples = []

In [ ]:
#EXAMPLES and DATASET
for index, row in csv_df.iterrows():
    sample_ith = SingleTurnSample(
        user_input=row['Question/Prompt'],
        retrieved_contexts=[row['Contexts']],
        response=row['Answer'],
        reference=row['Ground Truth'],
    )

    evaluation_samples.append(sample_ith)

In [ ]:
eval_dataset = EvaluationDataset(samples=evaluation_samples)

In [ ]:
eval_dataset

In [ ]:
#run_config = RunConfig(timeout=120)  # Set timeout to 60 seconds

In [ ]:
evaluator_llm = LangchainLLMWrapper(OllamaLLM(model="granite3-dense:8b", temperature=0.0))
metrics = [Faithfulness(), FactualCorrectness()]

In [ ]:
results = evaluate(dataset=eval_dataset, metrics=metrics, llm=evaluator_llm, raise_exceptions=False)

In [ ]:
results.to_pandas()